## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps



In [3]:
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\aryam\class\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pontian Kecil,MY,1.4801,103.3899,78.93,82,98,1.63,overcast clouds
1,1,Ambilobe,MG,-13.2000,49.0500,67.91,83,9,3.47,clear sky
2,2,Krasnoye,UA,49.9145,24.6134,51.19,56,14,11.52,few clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,44.71,57,0,34.52,clear sky
4,4,Ribeira Grande,PT,38.5167,-28.7000,71.83,89,24,6.46,few clouds


In [28]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 77
What is the maximum temperature you would like for your trip? 98


In [29]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [30]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                183
City                   183
Country                183
Lat                    183
Lng                    183
Max Temp               183
Humidity               183
Cloudiness             183
Wind Speed             183
Current Description    183
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pontian Kecil,MY,78.93,overcast clouds,1.4801,103.3899,
5,Del Rio,US,89.71,clear sky,29.3627,-100.8968,
11,Hilo,US,80.80,overcast clouds,19.7297,-155.0900,
12,Mecca,SA,86.04,few clouds,21.4267,39.8261,
16,Victoria,HK,84.33,overcast clouds,22.2855,114.1577,
23,Dyersburg,US,82.98,light rain,36.0345,-89.3856,
24,Atuona,PF,77.81,few clouds,-9.8000,-139.0333,
25,Boa Vista,BR,91.38,scattered clouds,2.8197,-60.6733,
29,Kapaa,US,80.58,moderate rain,22.0752,-159.3190,
31,Kavieng,PG,82.63,overcast clouds,-2.5744,150.7967,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

    

    
    
    

     
   
    
    
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()


City                   183
Country                183
Max Temp               183
Current Description    183
Lat                    183
Lng                    183
Hotel Name             183
dtype: int64

In [36]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current Description} and {Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# 11b. Display the figure
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.3)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))